In [ ]:
# !pip install git+https://github.com/deepmind/dm-haiku

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-ytemnkv8
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-ytemnkv8
  Created wheel for dm-haiku: filename=dm_haiku-0.0.7.dev0-py3-none-any.whl size=588272 sha256=6f9ee99874f3ad1357bed284c072cf2b02b71fe641e35f4e0a93d1c094d24434
  Stored in directory: /tmp/pip-ephem-wheel-cache-wq2e575j/wheels/06/28/69/ebaac5b2435641427299f29d88d005fb4e2627f4a108f0bdbc
Successfully built dm-haiku


In [ ]:
from typing import Iterator, Tuple, Mapping

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from typing import Any, NamedTuple

import haiku as hk
import jax
from jax import lax
import jax.numpy as jnp
import numpy as np
import tensorflow_datasets as tfds

from jax import random

In [ ]:
# import outfitprocessing as data

Mounted at /content/gdrive
94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
# OutfitBatchData = data.sequence_Tensor
# OutfitBatchData['outfitSequencesImage'].shape, OutfitBatchData['outfitSequencesCaption'].shape

((3, 8, 100352), (3, 8, 2756))

In [ ]:
# V = OutfitBatchData['outfitSequencesImage']
# S = OutfitBatchData['outfitSequencesCaption']

# images   = jnp.reshape(V, (-1, V.shape[-1]))
# captions = jnp.reshape(S, (-1, S.shape[-1]))

# images.shape, captions.shape

((24, 100352), (24, 2756))

In [ ]:
# captions_Len = jnp.reshape(jnp.sum(captions, axis=1), (-1,1))
# captions.shape, captions_Len.shape

((24, 2756), (24, 1))

In [ ]:
# VisualSemanticData = {'img_batch' : images, 'caption_batch' : captions/captions_Len}

In [ ]:
# VisualSemanticData['img_batch'].shape, VisualSemanticData['caption_batch'].shape, VisualSemanticData['img_batch'].dtype, VisualSemanticData['caption_batch'].dtype, type(VisualSemanticData['img_batch']), type(VisualSemanticData['caption_batch'])

((24, 100352),
 (24, 2756),
 dtype('float32'),
 dtype('float32'),
 jaxlib.xla_extension.DeviceArray,
 jaxlib.xla_extension.DeviceArray)

In [ ]:
class visual_semantic(hk.Module):

  def __init__(self, embedding_dimension, m, name=None):
    super().__init__(name=name)
    self.embedding_dimension = embedding_dimension
    self.m = m

  def __call__(self, vs):
    v = vs['img_batch']
    s = vs['caption_batch']

    i,j,k = v.shape[-1], self.embedding_dimension, s.shape[-1]

    wv_init = hk.initializers.TruncatedNormal(1. / np.sqrt(j))
    ws_init = hk.initializers.TruncatedNormal(1. / np.sqrt(j))

    wv = hk.get_parameter("wv", shape=[i, j], dtype=v.dtype, init=wv_init)
    ws = hk.get_parameter("ws", shape=[j, k], dtype=s.dtype, init=ws_init)

#  ve-->(NumberOfImages, EmbeddingSize) v-->(NumberOfImages, SizeOfImage) wv-->(SizeOfImage, EmbeddingSize)
    ve = jnp.matmul(v,wv)   
#  sT-->(SizeOfCaption, NumberOfCaptions) s-->(NumberOfCaptions, SizeOfCaption)
    sT = jnp.transpose(s)
#  se-->(EmbeddingSize, NumberOfCaptions) ws-->(EmbeddingSize, SizeOfCaption) sT-->(SizeOfCaption, NumberOfCaptions)    
    se = jnp.matmul(ws, sT)

    ve_m = jnp.reshape(jnp.linalg.norm(ve, axis=1), (-1,1))
    se_m = jnp.reshape(jnp.linalg.norm(se, axis=0), (1,-1))

    ve_u = ve/ve_m
    se_u = se/se_m

    v_CosMatrix_s = jnp.matmul(ve_u, se_u)
    vN, sN = v_CosMatrix_s.shape
    
    v_CosDiagonal_s = jnp.diagonal(v_CosMatrix_s)

    v_VARs = self.m+jnp.subtract(v_CosMatrix_s, jnp.reshape(v_CosDiagonal_s, (-1,1)))
    s_VARv = self.m+jnp.subtract(v_CosMatrix_s, jnp.reshape(v_CosDiagonal_s, (1,-1)))

    zeroMatrix = jnp.zeros_like(v_CosMatrix_s)

    v_VARs_max = jnp.maximum(v_VARs,zeroMatrix)
    s_VARv_max = jnp.maximum(s_VARv,zeroMatrix)

    v_VARs_final = jnp.subtract(v_VARs_max, jnp.diag(jnp.diag(v_VARs_max)))
    s_VARv_final = jnp.subtract(s_VARv_max, jnp.diag(jnp.diag(s_VARv_max)))

    return (jnp.sum(v_VARs_final)+jnp.sum(s_VARv_final))/(vN*sN) 

In [ ]:
def visual_semantic_fn(vs) -> jnp.ndarray:
  embeddingSize = 512
  m = .2
  model = visual_semantic(embeddingSize, m)
  return model(vs)

In [ ]:
# visual_semantic_pure_fn = hk.transform(visual_semantic_fn)

# key = hk.PRNGSequence(42)
# params = visual_semantic_pure_fn.init(next(key), VisualSemanticData)

# y = visual_semantic_pure_fn.apply(params, None, VisualSemanticData)
# y

DeviceArray(228.23099, dtype=float32)

In [ ]:
# params_init, loss_fn  = hk.without_apply_rng(hk.transform(visual_semantic_fn))
# loss_fn = jax.jit(loss_fn)

# rng = hk.PRNGSequence(42)
# initial_params = params_init(next(rng), VisualSemanticData)

# gradients = jax.grad(loss_fn)(initial_params, VisualSemanticData)
# gradients.keys()

# gradients['visual_semantic']

{'ws': DeviceArray([[ 0.        , -0.69538575,  0.        , ...,  0.        ,
                0.        ,  0.13365972],
              [ 0.        , -0.92906183,  0.        , ...,  0.        ,
                0.        ,  5.7322807 ],
              [ 0.        , -0.46656233,  0.        , ...,  0.        ,
                0.        ,  1.5838557 ],
              ...,
              [ 0.        ,  0.87496644,  0.        , ...,  0.        ,
                0.        ,  1.5654523 ],
              [ 0.        , -2.2239273 ,  0.        , ...,  0.        ,
                0.        ,  3.8095927 ],
              [ 0.        , -0.21322386,  0.        , ...,  0.        ,
                0.        , -1.3571486 ]], dtype=float32),
 'wv': DeviceArray([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
                0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
              [ 2.0631300e-02, -3.7829585e-03, -2.6160937e-02, ...,
               -1.1592135e-03,  5.8289450e-03, -1.9538224e-02],
    

TESTING

In [ ]:
# def visual_semantic_model_test(vs):
#   v = vs['img_seq']
#   s = vs['caption_seq']

#   embedding_dimension = 512
#   m = .2

#   i,j,k = v.shape[-1], embedding_dimension, s.shape[-1]

#   # wv_init = hk.initializers.TruncatedNormal(1. / np.sqrt(j))
#   # ws_init = hk.initializers.TruncatedNormal(1. / np.sqrt(j))

#   # wv = hk.get_parameter("wv", shape=[i, j], dtype=v.dtype, init=wv_init)
#   # ws = hk.get_parameter("ws", shape=[j, k], dtype=s.dtype, init=ws_init)

#   key1 = random.PRNGKey(7)
#   key2 = random.PRNGKey(11)

#   wv = random.normal(key1, shape=(i, j))
#   ws = random.normal(key2, shape=(j, k))

# #  ve-->(NumberOfImages, EmbeddingSize) v-->(NumberOfImages, SizeOfImage) wv-->(SizeOfImage, EmbeddingSize)
#   ve = jnp.matmul(v,wv)   
# #  sT-->(SizeOfCaption, NumberOfCaptions) s-->(NumberOfCaptions, SizeOfCaption)
#   sT = jnp.transpose(s)
# #  se-->(EmbeddingSize, NumberOfCaptions) ws-->(EmbeddingSize, SizeOfCaption) sT-->(SizeOfCaption, NumberOfCaptions)    
#   se = jnp.matmul(ws, sT)

#   ve_m = jnp.reshape(jnp.linalg.norm(ve, axis=1), (-1,1))
#   se_m = jnp.reshape(jnp.linalg.norm(se, axis=0), (1,-1))

#   ve_u = ve/ve_m
#   se_u = se/se_m

#   v_CosMatrix_s = jnp.matmul(ve_u, se_u)
#   v_CosDiagonal_s = jnp.diagonal(v_CosMatrix_s)

#   v_VARs = m+jnp.subtract(v_CosMatrix_s, jnp.reshape(v_CosDiagonal_s, (-1,1)))
#   s_VARv = m+jnp.subtract(v_CosMatrix_s, jnp.reshape(v_CosDiagonal_s, (1,-1)))

#   zeroMatrix = jnp.zeros_like(v_CosMatrix_s)

#   v_VARs_max = jnp.maximum(v_VARs,zeroMatrix)
#   s_VARv_max = jnp.maximum(s_VARv,zeroMatrix)

#   return jnp.sum(v_VARs_max)+jnp.sum(s_VARv_max)

In [ ]:
# visual_semantic_model_test(VisualSemanticData)

DeviceArray(40.47211, dtype=float32)

CACHE

In [ ]:
# dd = jnp.sum(VisualSemanticData['caption_seq'], axis=1)
# qq = [i for i in list(dd[1,:]) if i==1]
# qq

[DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32),
 DeviceArray(1, dtype=int32)]